# Data projet abritel

In [83]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [24]:
pip install openpyxl

     ---------------------------------------- 0.0/250.0 kB ? eta -:--:--
     ----------------------- -------------- 153.6/250.0 kB 4.6 MB/s eta 0:00:01
     -------------------------------------- 250.0/250.0 kB 3.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [27]:
pip install xlsxwriter

     ---------------------------------------- 0.0/152.7 kB ? eta -:--:--
     ----------------------------------- -- 143.4/152.7 kB 4.3 MB/s eta 0:00:01
     -------------------------------------- 152.7/152.7 kB 3.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [53]:
from IPython.display import display
import json
import pandas as pd
pd.options.display.max_columns = None # Display all columns of a dataframe
pd.options.display.max_rows = 700
from pprint import pprint
import re
import requests
from bs4 import BeautifulSoup

import time
import os
from selenium import webdriver
from selenium.webdriver.common. by import By

import numpy as np

#vérifier à la fin qu'on a bien utiliser ça
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

In [2]:
#Add following options before initializing the webdriver

# chromeOptions = webdriver.ChromeOptions()
# chromeOptions.add_argument("--headless")
# chromeOptions.add_argument("--remote-debugging-port=9222")
#chromeOptions.add_argument('--no-sandbox')

path="C:/Users/32498/Downloads/ECON2206-Data-Management-2023-main/chromedriver_win32"
driver = webdriver.Chrome(executable_path=os.path.join(path, 'chromedriver'))
driver.get('https://www.abritel.fr/search/keywords:belgique/arrival:2023-09-04/departure:2023-09-17/minNightlyPrice/0/minTotalPrice/0?filterByTotalPrice=true&petIncluded=false&ssr=true')
time.sleep(3)

#reject cookies
reject_all_button = driver.find_element(By.XPATH,"//button[contains(@class, 'osano-cm-denyAll')]")
reject_all_button.click()

time.sleep(2)


# 1. Web scraping part

In [3]:
base_url='https://www.abritel.fr/search/keywords:belgique/page:{}/arrival:2023-09-04/departure:2023-09-17/minNightlyPrice/0/minTotalPrice/0?filterByTotalPrice=true&petIncluded=false&ssr=true'

num_pages = 9

all_names = []
price_night_list=[]
price_total_list=[]
rating_info=[]
types=[]
features_list=[]
premium_list = []

for page_num in range(1, num_pages+1):
    link = base_url.format(page_num)
    driver.get(link)
    time.sleep(2) # attendre quelques secondes pour que la page se charge complètement
    
    for i in range(1, 22):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight / 22 * {});".format(i))
        time.sleep(1.5)
    
    # Extraire les noms de la page en cours
    names = driver.find_elements(By.CSS_SELECTOR, 'h2.HitInfo__headline')
    name_list = [name.text for name in names]
    all_names.extend(name_list)
    
    prices = driver.find_elements(By.CSS_SELECTOR, 'span.DualPrice__secondary')
    prices_night = [price.text if price.text else None for price in prices]
    price_night_list.extend(prices_night)

    prices_total = driver.find_elements(By.CSS_SELECTOR, 'span.DualPrice__primary')
    total_list = [price_total.text if price_total.text else None for price_total in prices_total]
    price_total_list.extend(total_list)

    type_houses=driver.find_elements(By.CSS_SELECTOR,'div.HitInfo__content div.HitInfo__type-place-details span:first-child')
    type_list=[type_house.text for type_house in type_houses]
    types.extend(type_list)

    elements = driver.find_elements(By.CSS_SELECTOR, 'div.HitCarousel') # trouver tous les éléments à vérifier
    for element in elements:
        premium = element.find_elements(By.CSS_SELECTOR, 'span.HitInfo__labels_item') # trouver tous les éléments "premium"
        if premium: # vérifier si l'élément "premium" existe
            premium_list.append('1') # si oui, ajouter 1
        else:
            premium_list.append('0') # sinon, ajouter 0

    
    infos=driver.find_elements(By.CSS_SELECTOR,'div.HitInfo__starRating')
    for info in infos:
        rating_num = '0'
        rating_count = '0 appréciation'
        rating_status = 'rien'
    
        rating_nums = info.find_elements(By.CSS_SELECTOR, 'span.Rating__label')
        if rating_nums:
            rating_num = rating_nums[0].text

        rating_counts = info.find_elements(By.CSS_SELECTOR, 'span.Rating__count')
        if rating_counts:
            rating_count = rating_counts[0].text

        rating_3 = info.find_elements(By.CSS_SELECTOR, 'span.HitInfo__ranked')
        if rating_3:
            rating_status = rating_3[0].text
        
        rating_info.append([rating_num, rating_count, rating_status])

    for rating in rating_info:
        rating=pd.DataFrame(rating_info, columns=["Rating", "Appreciations","Type of annoucement"])                   
    
    elements = driver.find_elements(By.CSS_SELECTOR, "div.HitInfo__room-beds-details")
    for element in elements:
        span_elements = element.find_elements(By.TAG_NAME, "span")
        personnes = re.sub("[^0-9]", "", span_elements[0].text) if span_elements else '0'
        chambres = re.sub("[^0-9]", "", span_elements[1].text) if len(span_elements) > 1 else '0'
        lits = re.sub("[^0-9]", "", span_elements[2].text) if len(span_elements) > 2 else "0"
        features_list.append([personnes,chambres,lits])

features = pd.DataFrame(features_list, columns=["Number of Guests", "Number of Bedrooms", "Number of Beds"])

driver.quit()

In [4]:
data = {
    'Name': all_names,
    'Type': types,
    'hote premium':premium_list
}
# Créer le dataframe à partir du dictionnaire
df1 = pd.DataFrame(data)
df1
# Afficher le dataframe

,Name,Type,hote premium
0,"95 m2 Appartement 5étage , 3 chambres, Wi-fi, ...",Appartement,1
1,"Grand jardin, moyenne montagne, à la campagne,...",Appartement,0
2,Bertogne: Ardennes Belges - Gite récent en bor...,Cottage,0
3,Chalet Zutendaal,Chalet,0
4,Ferme de Filot,Ferme,1
5,"Bel appartement pour 4 personnes avec WIFI, TV...",Appartement,0
6,Confortable gîte près du lac Robertville,Cottage,1
7,"Face au Parlement Européen, bel appartement",Appartement,1
8,COCOONING en bord de Meuse.,Cottage,1
9,Charmante maison de pêcheur avec jardin privé,Maison,1


In [5]:
data_price={
    'Price per Night': price_night_list,
    'Total Price': price_total_list,
}
dfp=pd.DataFrame(data_price)
dfp

,Price per Night,Total Price
0,122 €/nuit,1 592 €montant total
1,63 €/nuit,818 €montant total
2,74 €/nuit,957 €montant total
3,65 €/nuit,851 €montant total
4,67 €/nuit,870 €montant total
5,122 €/nuit,1 585 €montant total
6,72 €/nuit,938 €montant total
7,97 €/nuit,1 255 €montant total
8,108 €/nuit,1 408 €montant total
9,102 €/nuit,1 332 €montant total


In [58]:
df=pd.concat([df1, dfp, rating,features], axis=1)
df.set_index('Name', inplace=True)
df

,Type,hote premium,Price per Night,Total Price,Rating,Appreciations,Type of annoucement,Number of Guests,Number of Bedrooms,Number of Beds
Name,,,,,,,,,,
"95 m2 Appartement 5étage , 3 chambres, Wi-fi, Garage, Terrasse",Appartement,1,122 €/nuit,1 592 €montant total,4.5,(51 appréciations),Particulier,7,3,7
"Grand jardin, moyenne montagne, à la campagne,situation idyll. animaux acceptés",Appartement,0,63 €/nuit,818 €montant total,4.7,(58 appréciations),Particulier,4,1,3
Bertogne: Ardennes Belges - Gite récent en bordure de forêt,Cottage,0,74 €/nuit,957 €montant total,4.4,(81 appréciations),Particulier,6,3,7
Chalet Zutendaal,Chalet,0,65 €/nuit,851 €montant total,0,(0 appréciation),rien,4,2,0
Ferme de Filot,Ferme,1,67 €/nuit,870 €montant total,4.6,(17 appréciations),Particulier,4,1,2
"Bel appartement pour 4 personnes avec WIFI, TV et balcon",Appartement,0,122 €/nuit,1 585 €montant total,0,(0 appréciation),rien,4,2,0
Confortable gîte près du lac Robertville,Cottage,1,72 €/nuit,938 €montant total,0,(0 appréciation),rien,4,2,4
"Face au Parlement Européen, bel appartement",Appartement,1,97 €/nuit,1 255 €montant total,4.8,(66 appréciations),Particulier,5,1,2
COCOONING en bord de Meuse.,Cottage,1,108 €/nuit,1 408 €montant total,4.8,(13 appréciations),Particulier,3,1,1


In [59]:
df['Price per Night'] = df['Price per Night'].str.replace('€/nuit', '').str.replace(' ', '').astype(float)
df['Total Price'] = df['Total Price'].apply(lambda x: np.nan if pd.isna(x) or x == 'Consultez les détails pour voir le prix total' else float(x.replace(' €montant total', '').replace(' ', '').replace('€', '')))
df['Rating'] = df['Rating'].astype(float)
df['Appreciations'] = df['Appreciations'].str.extract('(\d+)').astype(float)

df['Number of Guests'] = df['Number of Guests'].astype(float)
df['Number of Bedrooms'] =df['Number of Bedrooms'].astype(float)
df['Number of Beds'] = df['Number of Beds'].astype(float)


In [60]:
df

,Type,hote premium,Price per Night,Total Price,Rating,Appreciations,Type of annoucement,Number of Guests,Number of Bedrooms,Number of Beds
Name,,,,,,,,,,
"95 m2 Appartement 5étage , 3 chambres, Wi-fi, Garage, Terrasse",Appartement,1,122.0,1592.0,4.5,51.0,Particulier,7.0,3.0,7.0
"Grand jardin, moyenne montagne, à la campagne,situation idyll. animaux acceptés",Appartement,0,63.0,818.0,4.7,58.0,Particulier,4.0,1.0,3.0
Bertogne: Ardennes Belges - Gite récent en bordure de forêt,Cottage,0,74.0,957.0,4.4,81.0,Particulier,6.0,3.0,7.0
Chalet Zutendaal,Chalet,0,65.0,851.0,0.0,0.0,rien,4.0,2.0,0.0
Ferme de Filot,Ferme,1,67.0,870.0,4.6,17.0,Particulier,4.0,1.0,2.0
"Bel appartement pour 4 personnes avec WIFI, TV et balcon",Appartement,0,122.0,1585.0,0.0,0.0,rien,4.0,2.0,0.0
Confortable gîte près du lac Robertville,Cottage,1,72.0,938.0,0.0,0.0,rien,4.0,2.0,4.0
"Face au Parlement Européen, bel appartement",Appartement,1,97.0,1255.0,4.8,66.0,Particulier,5.0,1.0,2.0
COCOONING en bord de Meuse.,Cottage,1,108.0,1408.0,4.8,13.0,Particulier,3.0,1.0,1.0


In [62]:
df.to_excel('Abriteldf.xlsx',index=True)

## Localisation

In [19]:
path="C:/Users/32498/Downloads/ECON2206-Data-Management-2023-main/chromedriver_win32"
driver = webdriver.Chrome(executable_path=os.path.join(path, 'chromedriver'))
driver.get('https://www.abritel.fr/search/keywords:belgique/arrival:2023-09-04/departure:2023-09-17/minNightlyPrice/0/minTotalPrice/0?filterByTotalPrice=true&petIncluded=false&ssr=true')
time.sleep(3)

#reject cookies
reject_all_button = driver.find_element(By.XPATH,"//button[contains(@class, 'osano-cm-denyAll')]")
reject_all_button.click()
time.sleep(2)

base_url='https://www.abritel.fr/search/keywords:belgique/page:{}/arrival:2023-09-04/departure:2023-09-17/minNightlyPrice/0/minTotalPrice/0?filterByTotalPrice=true&petIncluded=false&ssr=true'

num_pages = 9

loc_list = []
all_names = []

for page_num in range(1, num_pages+1):
    urls = []
    link = base_url.format(page_num)
    driver.get(link)
    time.sleep(2)
    
    for i in range(1, 22):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight / 22 * {});".format(i))
        time.sleep(1)
    
    annonces = driver.find_elements(By.CSS_SELECTOR, 'a.media-flex__content')
    # On boucle sur chaque annonce pour récupérer son URL et l'ajouter à la liste
    for annonce in annonces:
        url = annonce.get_attribute('href')
        urls.append(url)

    for url in urls:
        driver.get(url)
        time.sleep(2)
        localization = driver.find_element(By.CSS_SELECTOR,'a.u-hover-pointer.text-muted')
        localization_text = localization.text
        loc_list.append(localization_text)

        names = driver.find_elements(By.CSS_SELECTOR, 'h1.h2.margin-bottom-0x')
        name_list = [name.text for name in names]
        all_names.extend(name_list)

        driver.back()
        time.sleep(2)

KeyboardInterrupt: 

In [20]:
all_names= pd.DataFrame(all_names, columns=["Name"])
all_names

,Name
0,"95 m2 Appartement 5étage , 3 chambres, Wi-fi, ..."
1,"Grand jardin, moyenne montagne, à la campagne,..."
2,Bertogne: Ardennes Belges - Gite récent en bor...
3,Chalet Zutendaal
4,Ferme de Filot
5,Maison de ville Centre-ville de Gand
6,Confortable gîte près du lac Robertville
7,"Face au Parlement Européen, bel appartement"
8,COCOONING en bord de Meuse.
9,Charmante maison de pêcheur avec jardin privé


In [21]:
import pandas as pd
from geopy.geocoders import Nominatim

# Créer les listes vides pour stocker les valeurs de longitude et latitude
longitude = []
latitude = []

# Initialiser le géocodeur Nominatim
geolocator = Nominatim(user_agent="my_app")

# Parcourir la liste des lieux et récupérer les coordonnées géographiques pour chaque lieu
for location in loc_list:
    # Utiliser la fonction geocode de geopy pour récupérer les coordonnées géographiques de chaque lieu
    location = geolocator.geocode(location)
    # Si la fonction geocode a retourné une valeur, ajouter les valeurs de longitude et latitude aux listes
    if location is not None:
        longitude.append(location.longitude)
        latitude.append(location.latitude)
    # Si la fonction geocode n'a pas retourné de valeur, ajouter des valeurs NaN aux listes
    else:
        longitude.append(float("NaN"))
        latitude.append(float("NaN"))

# Créer un dataframe à partir des deux listes
dfloc = pd.DataFrame({'lieu': loc_list, 'latitude': latitude, 'longitude': longitude})

print(dfloc)

                        lieu   latitude   longitude
0               Koksijde-Bad  51.118739    2.634960
1                 Sankt-Vith  50.280388    6.125895
2                  Flamierge  50.060338    5.609939
3                  Zutendaal  50.931899    5.572555
4                      Filot  50.426751    5.568050
5          Ghent City Centre  43.742620  -79.295214
6                Robertville  47.696274  -65.773424
7           European Quarter  50.842350    4.383945
8                   Blaimont  44.302559    0.863393
9                  Adinkerke  51.075539    2.602209
10               Trois-Ponts  50.372681    5.870296
11                 Zutendaal  50.931899    5.572555
12                   Roumont  50.071405    5.559563
13        Historisch centrum  52.204378    6.801045
14                   Rendeux  50.214367    5.522222
15                  Waterloo  42.498275  -92.332874
16                 Viroinval  50.051636    4.627029
17        Ostend City Centre  50.112287    8.707017
18          

In [33]:
dfloc_total=pd.concat([all_names,dfloc], axis=1)
dfloc_total.set_index('Name', inplace=True)
dfloc_total

,lieu,latitude,longitude
Name,,,
"95 m2 Appartement 5étage , 3 chambres, Wi-fi, Garage, Terrasse",Koksijde-Bad,51.118739,2.634960
"Grand jardin, moyenne montagne, à la campagne,situation idyll. animaux acceptés",Sankt-Vith,50.280388,6.125895
Bertogne: Ardennes Belges - Gite récent en bordure de forêt,Flamierge,50.060338,5.609939
Chalet Zutendaal,Zutendaal,50.931899,5.572555
Ferme de Filot,Filot,50.426751,5.568050
Maison de ville Centre-ville de Gand,Ghent City Centre,43.742620,-79.295214
Confortable gîte près du lac Robertville,Robertville,47.696274,-65.773424
"Face au Parlement Européen, bel appartement",European Quarter,50.842350,4.383945
COCOONING en bord de Meuse.,Blaimont,44.302559,0.863393


In [63]:
dfloc_total.to_excel('Abritel_loc.xlsx',index=True)

## Merging of the two main dataframe

In [64]:
df_50 = df.head(50)

# Merge the two data frames on the "Name" column
merged_df = pd.merge(df_50,dfloc_total, on='Name')
merged_df

,Type,hote premium,Price per Night,Total Price,Rating,Appreciations,Type of annoucement,Number of Guests,Number of Bedrooms,Number of Beds,lieu,latitude,longitude
Name,,,,,,,,,,,,,
"95 m2 Appartement 5étage , 3 chambres, Wi-fi, Garage, Terrasse",Appartement,1,122.0,1592.0,4.5,51.0,Particulier,7.0,3.0,7.0,Koksijde-Bad,51.118739,2.634960
"Grand jardin, moyenne montagne, à la campagne,situation idyll. animaux acceptés",Appartement,0,63.0,818.0,4.7,58.0,Particulier,4.0,1.0,3.0,Sankt-Vith,50.280388,6.125895
Bertogne: Ardennes Belges - Gite récent en bordure de forêt,Cottage,0,74.0,957.0,4.4,81.0,Particulier,6.0,3.0,7.0,Flamierge,50.060338,5.609939
Chalet Zutendaal,Chalet,0,65.0,851.0,0.0,0.0,rien,4.0,2.0,0.0,Zutendaal,50.931899,5.572555
Chalet Zutendaal,Chalet,0,65.0,851.0,0.0,0.0,rien,4.0,2.0,0.0,Zutendaal,50.931899,5.572555
Chalet Zutendaal,Chalet,0,86.0,1118.0,0.0,0.0,rien,6.0,3.0,0.0,Zutendaal,50.931899,5.572555
Chalet Zutendaal,Chalet,0,86.0,1118.0,0.0,0.0,rien,6.0,3.0,0.0,Zutendaal,50.931899,5.572555
Ferme de Filot,Ferme,1,67.0,870.0,4.6,17.0,Particulier,4.0,1.0,2.0,Filot,50.426751,5.568050
Confortable gîte près du lac Robertville,Cottage,1,72.0,938.0,0.0,0.0,rien,4.0,2.0,4.0,Robertville,47.696274,-65.773424


In [65]:
merged_df.to_excel('Abritel_merged.xlsx',index=True)